In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
#from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import declarative_base, sessionmaker

In [2]:
engine = create_engine('sqlite:///empleados_orm.db', echo=True)

In [3]:
Base = declarative_base()


In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [10]:
class Empleado(Base):
    __tablename__ = 'empleados'

    id = Column(Integer, primary_key=True)
    nombre = Column(String)
    apellido = Column(String)
    edad = Column(Integer)
    salario = Column(Float)
    departamento = Column(String)

    def __repr__(self):
        return f"<Empleado(nombre='{self.nombre}', apellido='{self.apellido}', departamento='{self.departamento}')>"


In [11]:
Base.metadata.create_all(engine)

2024-08-18 13:01:06,589 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-18 13:01:06,590 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("empleados")
2024-08-18 13:01:06,590 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-18 13:01:06,591 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("empleados")
2024-08-18 13:01:06,591 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-18 13:01:06,591 INFO sqlalchemy.engine.Engine 
CREATE TABLE empleados (
	id INTEGER NOT NULL, 
	nombre VARCHAR, 
	apellido VARCHAR, 
	edad INTEGER, 
	salario FLOAT, 
	departamento VARCHAR, 
	PRIMARY KEY (id)
)


2024-08-18 13:01:06,592 INFO sqlalchemy.engine.Engine [no key 0.00021s] ()
2024-08-18 13:01:06,602 INFO sqlalchemy.engine.Engine COMMIT


In [12]:
nuevo_empleado = Empleado(nombre='Carlos', apellido='Méndez', edad=33, salario=62000, departamento='IT')
session.add(nuevo_empleado)
session.commit()


2024-08-18 13:02:35,668 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-18 13:02:35,672 INFO sqlalchemy.engine.Engine INSERT INTO empleados (nombre, apellido, edad, salario, departamento) VALUES (?, ?, ?, ?, ?)
2024-08-18 13:02:35,672 INFO sqlalchemy.engine.Engine [generated in 0.00069s] ('Carlos', 'Méndez', 33, 62000.0, 'IT')
2024-08-18 13:02:35,674 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
empleados_it = session.query(Empleado).filter_by(departamento='IT').all()
for empleado in empleados_it:
    print(empleado)

2024-08-18 13:03:37,653 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-18 13:03:37,655 INFO sqlalchemy.engine.Engine SELECT empleados.id AS empleados_id, empleados.nombre AS empleados_nombre, empleados.apellido AS empleados_apellido, empleados.edad AS empleados_edad, empleados.salario AS empleados_salario, empleados.departamento AS empleados_departamento 
FROM empleados 
WHERE empleados.departamento = ?
2024-08-18 13:03:37,656 INFO sqlalchemy.engine.Engine [generated in 0.00071s] ('IT',)
<Empleado(nombre='Carlos', apellido='Méndez', departamento='IT')>


In [14]:
empleado = session.query(Empleado).filter_by(nombre='Carlos').first()
empleado.salario = 65000
session.commit()


2024-08-18 13:04:42,612 INFO sqlalchemy.engine.Engine SELECT empleados.id AS empleados_id, empleados.nombre AS empleados_nombre, empleados.apellido AS empleados_apellido, empleados.edad AS empleados_edad, empleados.salario AS empleados_salario, empleados.departamento AS empleados_departamento 
FROM empleados 
WHERE empleados.nombre = ?
 LIMIT ? OFFSET ?
2024-08-18 13:04:42,613 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ('Carlos', 1, 0)
2024-08-18 13:04:42,615 INFO sqlalchemy.engine.Engine UPDATE empleados SET salario=? WHERE empleados.id = ?
2024-08-18 13:04:42,615 INFO sqlalchemy.engine.Engine [generated in 0.00053s] (65000.0, 1)
2024-08-18 13:04:42,617 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
session.delete(empleado)
session.commit()

2024-08-18 13:05:47,315 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-18 13:05:47,317 INFO sqlalchemy.engine.Engine SELECT empleados.id AS empleados_id, empleados.nombre AS empleados_nombre, empleados.apellido AS empleados_apellido, empleados.edad AS empleados_edad, empleados.salario AS empleados_salario, empleados.departamento AS empleados_departamento 
FROM empleados 
WHERE empleados.id = ?
2024-08-18 13:05:47,318 INFO sqlalchemy.engine.Engine [generated in 0.00108s] (1,)
2024-08-18 13:05:47,319 INFO sqlalchemy.engine.Engine DELETE FROM empleados WHERE empleados.id = ?
2024-08-18 13:05:47,320 INFO sqlalchemy.engine.Engine [generated in 0.00049s] (1,)
2024-08-18 13:05:47,321 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
from sqlalchemy.orm import relationship


In [6]:
class Departamento(Base):
    __tablename__ = 'departamentos'

    id = Column(Integer, primary_key=True)
    nombre = Column(String)
    empleados = relationship("Empleado", back_populates="departamento")

In [7]:
class Empleado(Base):
    __tablename__ = 'empleados'

    id = Column(Integer, primary_key=True)
    nombre = Column(String)
    departamento_id = Column(Integer, ForeignKey('departamentos.id', extend_existing=True))
    departamento = relationship("Departamento", back_populates="empleados")


/tmp/ipykernel_83317/1293045433.py:1: SAWarning: Can't validate argument 'extend_existing'; can't locate any SQLAlchemy dialect named 'extend'
  class Empleado(Base):


In [8]:
departamento = session.query(Departamento).filter_by(nombre='IT').first()
for empleado in departamento.empleados:
    print(empleado)

2024-08-18 13:12:57,550 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-18 13:12:57,553 INFO sqlalchemy.engine.Engine SELECT departamentos.id AS departamentos_id, departamentos.nombre AS departamentos_nombre 
FROM departamentos 
WHERE departamentos.nombre = ?
 LIMIT ? OFFSET ?
2024-08-18 13:12:57,554 INFO sqlalchemy.engine.Engine [generated in 0.00069s] ('IT', 1, 0)


OperationalError: (sqlite3.OperationalError) no such table: departamentos
[SQL: SELECT departamentos.id AS departamentos_id, departamentos.nombre AS departamentos_nombre 
FROM departamentos 
WHERE departamentos.nombre = ?
 LIMIT ? OFFSET ?]
[parameters: ('IT', 1, 0)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)